#Regressions

In [42]:
#Libraries that we are goint to use:
import pandas as pd
#!pip install statsmodels

import statsmodels.api as sm
import statsmodels.formula.api as smf


In [43]:
df= pd.read_csv("/Users/polgarcia/Documents/Master/T2/Text_Mining/H1(scrapping+D&D)/TextMininingHW1/final_cleaned_data.csv")

In [44]:
print(df.head()) 

                                     Name  Barcelona  Price  Rating  stars  \
0                        Isabella's House          1   2142    8.58    4.0   
1                         Sonder Casa Luz          1   4870    8.48    4.0   
2  Axel TWO Barcelona 4 Sup - Adults Only          1   2149    8.38    4.0   
3                           Acta Voraport          1   1540    8.88    3.0   
4                         Tembo Barcelona          1   2840    8.88    4.0   

   num_coments                                        Description  \
0        1.390  Isabella's House se encuentra a 2,9 km de Pase...   
1        2.889  El Sonder Casa Luz se encuentra en Barcelona y...   
2        2.742  El TWO Hotel Barcelona by Axel está situado en...   
3       13.850  Acta Voraport está en Barcelona, a 10 min a pi...   
4        2.697  Tembo Barcelona está en Barcelona, a 16 min a ...   

   During_Treatment                                Cleaned_Description  \
0                 1  isabell hous encuentr

## Explanation of the Dataset

- **`Name`**: The name of the hotel.  
- **`Barcelona`**: This column can take only two values:  
  - `0` if the hotel is not in Barcelona but in our control city, which in this case is Madrid (chosen due to its price similarity).  
  - `1` if the hotel is located in Barcelona.  
- **`Price`**: The price of the hotel in Euros.  
- **`Rating`**: The rating of the hotel on the booking website.  
- **`Description`**: The original hotel description as found on the website.  
- **`During_Treatment`**: This column can take two values:  
  - `0` if the recorded prices are from the control week (two weeks after the event).  
  - `1` if the prices correspond to the week of the event.  
- **`Cleaned_Description`**: The hotel description after text preprocessing.  


## Part 3.a
Write down a fixed effects regression equation that allows you to derive a difference-
in-difference estimate of the effect of the event on prices. Think of controls to
add, why is this relevant? Explain why you need a second city for this.

First lets run the most basic regression:

In [47]:

# Create the interaction term (DiD variable)
df["Barcelona_x_treatment"] = df["Barcelona"] * df["During_Treatment"]

#Fist we will run a regression without CONTROLS.

model = smf.ols(
    "Price ~ During_Treatment + Barcelona + Barcelona_x_treatment ", 
    data=df
).fit()

# Print summary results
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.359
Model:                            OLS   Adj. R-squared:                  0.358
Method:                 Least Squares   F-statistic:                     276.5
Date:                Tue, 04 Feb 2025   Prob (F-statistic):          1.95e-142
Time:                        22:20:07   Log-Likelihood:                -12038.
No. Observations:                1482   AIC:                         2.408e+04
Df Residuals:                    1478   BIC:                         2.410e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              1341.54

What can we take from here: **#The event significantly increases hotel prices in Barcelona (Barcelona_x_treatment = 1476.07, p < 0.001).**

### **Think of controls to add, why is this relevant? Explain why you need a second city for this.**

R: Controls we can  add:

IMPORTANT, We may have to CHANGE THAT, because the rating and the stars we are using it  for the 3c

 -  The rating in the web.  
 - The Number of stars of the hotels, as the hotels with more stars tend to have higher price, 

 - The distance to the center. The hotels that are more close tho the event/the center of the city ( the event is not done exacly in the center of the city but in the center are very good comunications to go to the event).

 - The number of coments that each hotel has in the webside .... (add more explanation)


In [49]:
#we will add the number of comments and the stars of the hotels. The distance to the center we didnt add because we had some problems with the Madrid dataset.add

model2 = smf.ols(
    "Price ~ During_Treatment + Barcelona + Barcelona_x_treatment + Rating + num_coments ", 
    data=df
).fit()

# Print summary results
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.408
Model:                            OLS   Adj. R-squared:                  0.406
Method:                 Least Squares   F-statistic:                     203.5
Date:                Tue, 04 Feb 2025   Prob (F-statistic):          3.92e-165
Time:                        22:22:09   Log-Likelihood:                -11964.
No. Observations:                1480   AIC:                         2.394e+04
Df Residuals:                    1474   BIC:                         2.397e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              -717.63

## 3.b


### How would you use text features from the description as controls? Think about the text in the descriptions you scraped. How would this help? Why would termslike "Barcelona" not help?

Use new controls as: 

    - Sentiment score. 


    - Mentions of key amenities (e.g.: "centro", "playa", "aeropuerto", "wifi", "metro", "piscina" )


    - Word_count as a proxy for hotel reputation.


In [ ]:
#We will use the 

In [50]:
#HERE the data frame is not the same that we had because now we will have the columns of the frequency of the data
model2=smf.ols(
    "Price ~ During_Treatment + Barcelona + Barcelona_x_treatment  + Word_Count + Sentence_Count +  Avg_Word_Length + Special_Mentions + Sentiment_Score",
    data=df
).fit()

print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.375
Model:                            OLS   Adj. R-squared:                  0.372
Method:                 Least Squares   F-statistic:                     126.4
Date:                Tue, 04 Feb 2025   Prob (F-statistic):          1.35e-145
Time:                        22:24:19   Log-Likelihood:                -12019.
No. Observations:                1482   AIC:                         2.405e+04
Df Residuals:                    1474   BIC:                         2.410e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept               300.58

- **Why Bacelona not help?**  : Almost Every hotel in the barcelona  mention "Barcelona" and making it non-informative. the word Barcelona doesn’t add variation to the model.

## 3.c


Now suppose we want to decompose the treatment effect by hotel quality. Can
you use the text description here? How would you use them to study heterogenous
treatment effects? Write down a regression equation.

WE can add another column that represents the quality. 
In the part 2 ee can leveraged text descriptions of the hotels to extract meaningful features related to quality.
Some examples were the apearance of this words luxury_words = ["spa","lujoso","lujoso", "exclusivo", "premium","sofisticado","personalizado","masajes", "sauna"]


WE use the number of stars of the hotels because we consider that as more stars a hotel has, more quality.

In [51]:
model2 = smf.ols(
    "Price ~ During_Treatment + Barcelona + Barcelona_x_treatment + Rating + stars + Luxury_Score ", 
    data=df
).fit()

# Print summary results
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.493
Model:                            OLS   Adj. R-squared:                  0.491
Method:                 Least Squares   F-statistic:                     238.9
Date:                Tue, 04 Feb 2025   Prob (F-statistic):          2.59e-213
Time:                        22:29:58   Log-Likelihood:                -11849.
No. Observations:                1480   AIC:                         2.371e+04
Df Residuals:                    1473   BIC:                         2.375e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              -880.51